## Bloque Feature engineering

En este notebook trabajamos con la generacion de nuevas features en base a las columnas que se decidio
que eran de utilidad.

Ademas tratamos los outliers y transformamos lo categorico a numerico


In [2]:
import re
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
%matplotlib inline

In [3]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",40)
sns.set(style="whitegrid")

In [4]:
df = pd.read_csv("path-a-los-datos" , sep = "|")


#### Eliminamos las columnas que definimos innecesarias en la exploracion de datos

In [5]:
# columnas a eliminar por defecto
col_defecto = [col for col in df.columns.tolist() if 
                                        ("EXPDTR_DATA_ARPU_AMT" in col or "SMS_OFFNET_EXP_ARPU_AMT" in col 
                                        or  "SPNDG_VOI_ONNET_ARPU" in col  or  "USE_LCL_VOI_AMT" in col
                                        or  "SPNDG_VOI_INTRNTL_ARPU" in col  or  "SPNDG_VOI_OFFNET_ARPU" in col
                                        or  "SMS_ONNET_EXP_ARPU_AMT" in col)]


# columnas a eliminar que contienen nulos
cant_nulos = df.isnull().sum()[df.isnull().sum() > 0]
col_nulos = cant_nulos.index.to_list()


# columnas extras a eliminar por analisis y no brindan info para el target
col_extras = ["SEGMENTATION", "MICROSEGMENTATION","SOURCE", "TENURE_CUSTOMER_BL", "PREP_RECH_CHNNL_MODE12W",
              "MSISDN", 'PREP_RECH_LAST_DAY_12W']


# columnas referidas al trafico de datos
col_trd =[]
#col_trd = ["TRD_M1", "TRD_M2", "TRD_M3", "TRD_STR_M1", "TRD_STR_M2", "TRD_STR_M3", "TRD_SN_M1", "TRD_SN_M2",
#           "TRD_SN_M3", "TRD_IM_M1", "TRD_IM_M2", "TRD_IM_M3", "TRD_OTH_M1", "TRD_OTH_M2", "TRD_OTH_M3"]


# columnas de pack de voz (tanto mensual como semanal)
col_voz = re.findall("\S*VOICE\S*"," ".join(df.columns.to_list()))


# columnas de pack de sms (tanto mensual como semanal) y trafico de sms
col_sms = re.findall("\S*SMS\S*"," ".join(df.columns.to_list()))


# columnas de la expiracion del pack de datos
col_exp = []
#col_exp = re.findall("\S*DATA_EXP\S*"," ".join(df.columns.to_list()))

# columnas de las recargas acumuladas por dia
col_dias = re.findall("\S*Q\S*[0-9]+W"," ".join(df.columns.to_list()))

# columnas del trafico de voz
col_trv = re.findall("TRV\S*"," ".join(df.columns.to_list()))

# columnas del df target
col_tg = re.findall("\S*W_[0-9]", " ".join(df.columns.to_list()))
col_tg.append('FLAG_MIG')

# ponemos todas las columnas en la misma lista
lista_elim = col_defecto + col_nulos + col_extras + col_trd + col_voz + col_sms + col_exp + col_dias + col_trv + col_tg
lista_elim = list(set(lista_elim))

# eliminamos las columnas de nuestro df
df = df.drop(lista_elim,axis=1).copy()
        
print(f"Se eliminaron {len(lista_elim)} columnas")
print(f"Ahora el dataframe tiene {df.shape[1]} columnas")

Se eliminaron 123 columnas
Ahora el dataframe tiene 103 columnas


#### Carga dataset con el target

In [6]:
df.head()

,SUSCRIBER_KEY,FECHA_CORTE,VALUE_SEGMENT,COMMERCIAL_ACTIVATION_DATE,TENURE_CUSTOMER,PREP_RECH_Q_EVT_X1,PREP_RECH_Q_EVT_X2,PREP_RECH_Q_EVT_X3,PREP_RECH_AMT_X1,PREP_RECH_AMT_X2,PREP_RECH_AMT_X3,PREP_RECH_NDAYS_LASTRECH_12W,PREP_RECH_Q_EVT_W1,PREP_RECH_Q_EVT_W2,PREP_RECH_Q_EVT_W3,PREP_RECH_Q_EVT_W4,PREP_RECH_Q_EVT_W5,PREP_RECH_Q_EVT_W6,PREP_RECH_Q_EVT_W7,PREP_RECH_Q_EVT_W8,PREP_RECH_Q_EVT_W9,PREP_RECH_Q_EVT_W10,PREP_RECH_Q_EVT_W11,PREP_RECH_Q_EVT_W12,PREP_RECH_AMT_W1,PREP_RECH_AMT_W2,PREP_RECH_AMT_W3,PREP_RECH_AMT_W4,PREP_RECH_AMT_W5,PREP_RECH_AMT_W6,PREP_RECH_AMT_W7,PREP_RECH_AMT_W8,PREP_RECH_AMT_W9,PREP_RECH_AMT_W10,PREP_RECH_AMT_W11,PREP_RECH_AMT_W12,TRD_M1,TRD_M2,TRD_M3,TRD_STR_M1,TRD_STR_M2,TRD_STR_M3,TRD_SN_M1,TRD_SN_M2,TRD_SN_M3,TRD_IM_M1,TRD_IM_M2,TRD_IM_M3,TRD_OTH_M1,TRD_OTH_M2,TRD_OTH_M3,PACK_DATA_Q_X1,PACK_DATA_Q_X2,PACK_DATA_Q_X3,PACK_DATA_AMT_X1,PACK_DATA_AMT_X2,PACK_DATA_AMT_X3,PACK_DATA_Q_W1,PACK_DATA_Q_W2,PACK_DATA_Q_W3,PACK_DATA_Q_W4,PACK_DATA_Q_W5,PACK_DATA_Q_W6,PACK_DATA_Q_W7,PACK_DATA_Q_W8,PACK_DATA_Q_W9,PACK_DATA_Q_W10,PACK_DATA_Q_W11,PACK_DATA_Q_W12,PACK_DATA_AMT_W1,PACK_DATA_AMT_W2,PACK_DATA_AMT_W3,PACK_DATA_AMT_W4,PACK_DATA_AMT_W5,PACK_DATA_AMT_W6,PACK_DATA_AMT_W7,PACK_DATA_AMT_W8,PACK_DATA_AMT_W9,PACK_DATA_AMT_W10,PACK_DATA_AMT_W11,PACK_DATA_AMT_W12,PACK_DATA_EXP_W1,PACK_DATA_EXP_W2,PACK_DATA_EXP_W3,PACK_DATA_EXP_W4,PACK_DATA_EXP_W5,PACK_DATA_EXP_W6,PACK_DATA_EXP_W7,PACK_DATA_EXP_W8,PACK_DATA_EXP_W9,PACK_DATA_EXP_W10,PACK_DATA_EXP_W11,PACK_DATA_EXP_W12,FLAG_MIG,PREP_RECH_AMT_W_0,PREP_RECH_AMT_W_1,PREP_RECH_AMT_W_2,PREP_RECH_AMT_W_3,PREP_RECH_Q_EVT_W_0,PREP_RECH_Q_EVT_W_1,PREP_RECH_Q_EVT_W_2,PREP_RECH_Q_EVT_W_3,target
0,CW_DPRE_65306301_20181124,08-SEP-19,2 - GOLD,24-NOV-18,9,1,4,6,5.0,16.5,19.5,9,0,1,0,0,2,0,1,1,2,2,1,1,0.0,5.0,0.0,0.0,6.5,0.0,5.0,5.0,7.0,4.5,3.0,5.0,8.76,23.49,18.98,7170,19564,14888,89,95,308,446,544,700,-7696.24,-20179.51,-15877.02,1,3,6,5.00,15.00,16.48,0,1,0,0,1,0,1,1,2,2,1,1,0.0,5.0,0.0,0.00,5.00,0.00,5.0,5.00,6.5,3.99,0.99,5.0,4,11,0,5,12,5,12,11,10,5,7,12,0,0.0,3.0,3.0,6.0,0,1,1,2,0
1,CW_DPRE_66645894_20150701,08-SEP-19,2 - GOLD,01-JUL-15,50,3,3,2,13.0,9.0,6.0,2,1,1,0,1,1,1,0,1,1,0,1,0,3.0,5.0,0.0,5.0,3.0,3.0,0.0,3.0,3.0,0.0,3.0,0.0,2.88,1.94,3.08,0,0,0,12,8,8,3028,1979,3242,-3037.12,-1985.06,-3246.92,3,3,2,12.00,9.00,6.00,1,1,0,1,1,1,0,1,1,0,1,0,3.0,4.5,0.0,4.50,3.00,3.00,0.0,3.00,3.0,0.00,3.00,0.0,9,9,2,9,9,7,2,9,8,3,10,2,0,0.0,5.0,5.0,0.0,0,1,1,0,0
2,CW_DPRE_65306080_20160428,08-SEP-19,3 - SILVER,28-APR-16,40,3,1,0,15.0,3.0,0.0,11,0,1,1,1,0,1,0,0,0,0,0,0,0.0,5.0,5.0,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,25.81,0.13,0.00,6026,105,0,10595,0,0,5110,24,0,-21705.19,-128.87,0.00,3,1,0,15.00,1.99,0.00,0,1,1,1,0,1,0,0,0,0,0,0,0.0,5.0,5.0,5.00,0.00,1.99,0.0,0.00,0.0,0.00,0.00,0.0,2,9,9,7,0,4,0,0,0,0,0,0,0,0.0,5.0,10.0,0.0,0,1,2,0,0
3,CW_DPRE_67558747_20181225,08-SEP-19,3 - SILVER,25-DEC-18,8,1,1,4,5.0,5.0,17.0,24,0,0,0,1,0,0,0,1,1,1,1,1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,2.0,5.0,7.25,14.09,9.18,2996,1299,1254,2038,3795,3386,488,1135,933,-5514.75,-6214.91,-5563.82,1,1,4,5.00,5.00,15.50,0,0,0,1,0,0,0,1,1,1,1,1,0.0,0.0,0.0,5.00,0.00,0.00,0.0,5.00,5.0,4.00,1.50,5.0,0,0,3,10,0,0,3,10,10,8,10,12,0,0.0,0.0,6.5,0.0,0,0,2,0,0
4,CW_DPRE_69672979_20150701,08-SEP-19,1 - PLATINUM,01-JUL-15,50,11,10,4,31.0,25.0,16.0,0,3,2,2,4,3,3,1,3,0,1,3,0,9.0,6.0,6.0,10.0,8.0,7.0,3.0,7.0,0.0,5.0,11.0,0.0,25.33,14.27,12.16,13579,7012,4209,3121,1415,1372,4228,2562,3444,-20902.67,-10974.73,-9012.84,11,9,4,30.98,22.96,16.00,3,2,2,4,3,2,1,3,0,1,3,0,9.0,6.0,6.0,9.98,7.99,4.99,3.0,6.98,0.0,5.00,11.00,0.0,9,7,7,8,8,8,5,9,5,12,12,6,0,6.0,8.0,6.0,6.0,2,3,2,2,0


In [7]:
df.shape

(870606, 103)

### Tratamiento de outliers

In [8]:
# vamos a tratar los outliers de los campos de y de datos
# en base a los graficos de bigotes realizados durante la exploracion de datos, vemos que hay outliers muy
# lejanos a la mediana


# Como solucion general se propone hacer lo siguiente para cada columna con outliers
# Obtenemos el intervalo en el que se mueven los outliers de la columna [minimo,maximo]
# De este intervalo encontramos el valor que se encuentra a una distancia de 2/3 del intervalo con respecto
# al minimo del mismo, a este valor lo llamamos limite.
# A todos los outliers con un valor superior al limite vamos a reemplazarlos por la media de los outliers


lista_col_outliers = re.findall("P\S*"," ".join(df.columns.to_list()))

#lista_col_outliers.remove("PREP_RECH_LAST_DAY_12W")
lista_col_outliers.remove("PREP_RECH_NDAYS_LASTRECH_12W")

for col_name in lista_col_outliers:
    
    q1 = df[col_name].quantile(0.25)
    q3 = df[col_name].quantile(0.75)
    iqr = q3 - q1
    minimo = q3 + 1.5 * iqr
    maximo = df[col_name].max()
    limite = ((maximo - minimo) * 2/3) + minimo
    
    mask = df[col_name] > minimo
    media = round(df[col_name][mask].values.mean(),2)

    df[col_name] = df[col_name].map(lambda x: x if x<=limite else media)


### Feature engineering

In [10]:
# creamos la nueva feature que relaciona las expiraciones de los datos y luego eliminamos esas features
col_exp = re.findall("\S*DATA_EXP\S*"," ".join(df.columns.to_list()))
df["DATA_EXP_MEAN"] = (df[col_exp].sum(axis=1)) / len(col_exp)

df = df.drop(col_exp,axis=1).copy()


In [11]:
# Tenemos variables mensuales y semanales, hay que quedarnos con alguna de ellas y hacer una transformacion de
# las otras.
# Como solucion vamos a quedarnos con las variables mensuales de cantidad de eventos, a las variables mensuales
# de monto de eventos vamos a transformarlas para volverlas adimensionales.


In [12]:
# Entonces:

# Campos a mantener intactos: 
#'PREP_RECH_Q_EVT_X1','PREP_RECH_Q_EVT_X2','PREP_RECH_Q_EVT_X3','PACK_DATA_Q_X1',
#'PACK_DATA_Q_X2','PACK_DATA_Q_X3'



# COLUMNA DE MONTO ANUAL RECARGAS
df['PREP_RECH_AMT_ANU'] = df['PREP_RECH_AMT_X1'] + df['PREP_RECH_AMT_X2'] + df['PREP_RECH_AMT_X3']
# Montos relativos por mes RECARGAS
df['PREP_RECH_AMT_REL_X1'] = (df['PREP_RECH_AMT_X1']/df['PREP_RECH_AMT_ANU']).fillna(0)
df['PREP_RECH_AMT_REL_X2'] = (df['PREP_RECH_AMT_X2']/df['PREP_RECH_AMT_ANU']).fillna(0)
df['PREP_RECH_AMT_REL_X3'] = (df['PREP_RECH_AMT_X3']/df['PREP_RECH_AMT_ANU']).fillna(0)

# COLUMNA DE MONTO ANUAL DATOS
df['PACK_DATA_AMT_ANU'] = df['PACK_DATA_AMT_X1'] + df['PACK_DATA_AMT_X2'] + df['PACK_DATA_AMT_X3']
# Montos relativos por mes DATOS
df['PACK_DATA_AMT_REL_X1'] = (df['PACK_DATA_AMT_X1']/df['PACK_DATA_AMT_ANU']).fillna(0)
df['PACK_DATA_AMT_REL_X2'] = (df['PACK_DATA_AMT_X2']/df['PACK_DATA_AMT_ANU']).fillna(0)
df['PACK_DATA_AMT_REL_X3'] = (df['PACK_DATA_AMT_X3']/df['PACK_DATA_AMT_ANU']).fillna(0)

# hay casos en los que el divisor es cero, a estas cuentas el resultado va a ser un NaN, por lo que los 
# reemplazamos por cero

# Eliminamos las columnas de los montos totales
df = df.drop(['PREP_RECH_AMT_ANU','PACK_DATA_AMT_ANU'],axis=1).copy()


# En cuanto a las variables semanales vamos a descartarlas por el momento
col_semanas = re.findall("\S*W[0-9]+"," ".join(df.columns.to_list()))
df = df.drop(col_semanas,axis=1).copy()


In [13]:
# Agregamos columnas relativas de cantidad de meses

# COLUMNA mes dos con respecto al mes 1 y 2 recargas
df['PREP_RECH_Q_REL_X2'] = df['PREP_RECH_Q_EVT_X2']/ (df['PREP_RECH_Q_EVT_X1'] + df['PREP_RECH_Q_EVT_X2'])
df['PREP_RECH_Q_REL_X2'] = df['PREP_RECH_Q_REL_X2'].fillna(0)
df['PREP_RECH_Q_REL_X2'] = df['PREP_RECH_Q_REL_X2'].map(lambda x: x if x != np.inf else 0)

# COLUMNA mes tres con respecto al total recargas
df['PREP_RECH_Q_REL_X3'] = df['PREP_RECH_Q_EVT_X3']/ (df['PREP_RECH_Q_EVT_X1'] + df['PREP_RECH_Q_EVT_X2'] + df['PREP_RECH_Q_EVT_X3'])
df['PREP_RECH_Q_REL_X3'] = df['PREP_RECH_Q_REL_X3'].fillna(0)
df['PREP_RECH_Q_REL_X3'] = df['PREP_RECH_Q_REL_X3'].map(lambda x: x if x != np.inf else 0)

# COLUMNA mes dos con respecto al mes 1 y 2 datos
df['PACK_DATA_Q_REL_X2'] = df['PACK_DATA_Q_X2']/ (df['PACK_DATA_Q_X1'] + df['PACK_DATA_Q_X2'])
df['PACK_DATA_Q_REL_X2'] = df['PACK_DATA_Q_REL_X2'].fillna(0)
df['PACK_DATA_Q_REL_X2'] = df['PACK_DATA_Q_REL_X2'].map(lambda x: x if x != np.inf else 0)

# COLUMNA mes tres con respecto al total DATOS
df['PACK_DATA_Q_REL_X3'] = df['PACK_DATA_Q_X3']/ (df['PACK_DATA_Q_X1'] + df['PACK_DATA_Q_X2'] + df['PACK_DATA_Q_X3'])
df['PACK_DATA_Q_REL_X3'] = df['PACK_DATA_Q_REL_X3'].fillna(0)
df['PACK_DATA_Q_REL_X3'] = df['PACK_DATA_Q_REL_X3'].map(lambda x: x if x != np.inf else 0)

In [14]:
# otras features interesantes se pueden obtener de la relacion que hay entre el monto y la cantidad de eventos
# de las recargas y los datos

# hay casos en los que el divisor es cero, a estas cuentas el resultado va a ser un NaN, por lo que los 
# reemplazamos por cero

df['PREP_RECH_REL_X1'] = (df['PREP_RECH_AMT_X1']/df['PREP_RECH_Q_EVT_X1']).fillna(0)
df['PREP_RECH_REL_X2'] = (df['PREP_RECH_AMT_X2']/df['PREP_RECH_Q_EVT_X2']).fillna(0)
df['PREP_RECH_REL_X3'] = (df['PREP_RECH_AMT_X3']/df['PREP_RECH_Q_EVT_X3']).fillna(0)

df['PACK_DATA_REL_X1'] = (df['PACK_DATA_AMT_X1']/df['PACK_DATA_Q_X1']).fillna(0)
df['PACK_DATA_REL_X2'] = (df['PACK_DATA_AMT_X2']/df['PACK_DATA_Q_X2']).fillna(0)
df['PACK_DATA_REL_X3'] = (df['PACK_DATA_AMT_X3']/df['PACK_DATA_Q_X3']).fillna(0)

col_montos = re.findall("\S*AMT_X[0-9]+"," ".join(df.columns.to_list()))
df = df.drop(col_montos,axis=1).copy()

In [15]:
# variables de tiempo, obtener la diferencia en dias entre la fecha de alta y la de cierre
# luego obtener un porcentaje en relacion al tiempo total

# Porcentaje del tiempo entre el primer evento y el último que se tiene registro


#creo columna nueva con la diferencia en dias entre las fechas de activacion y de corte
df["DAYS_CUSTOMER"] = (pd.to_datetime(df["FECHA_CORTE"])
                       - pd.to_datetime(df['COMMERCIAL_ACTIVATION_DATE'])).dt.days

# PREP_RECH_NDAYS_LASTRECH_12W dias desde la ultima recarga
df["PERCENT_DAYS"] = (df["DAYS_CUSTOMER"] - df["PREP_RECH_NDAYS_LASTRECH_12W"]) / df["DAYS_CUSTOMER"]
# indica el porcentaje de la antiguedad del cliente que representa su ultimo dia de actividad registrada

# con estas 2 nuevas columnas ya no necesito las otras columnas relacionadas con los tiempos
# las elimino del datafrane
df = df.drop(["PREP_RECH_NDAYS_LASTRECH_12W","FECHA_CORTE", 'COMMERCIAL_ACTIVATION_DATE', 'TENURE_CUSTOMER']
             , axis=1).copy()


In [17]:
# en cuanto al trafico de datos vamo a tomar solamente las features mensuales, al resto las tiramos

# columnas referidas al trafico de datos
col_trd = ["TRD_STR_M1", "TRD_STR_M2", "TRD_STR_M3", "TRD_SN_M1", "TRD_SN_M2",
           "TRD_SN_M3", "TRD_IM_M1", "TRD_IM_M2", "TRD_IM_M3", "TRD_OTH_M1", "TRD_OTH_M2", "TRD_OTH_M3"]

df = df.drop(col_trd,axis=1).copy()

#### Tratamiento de campos categoricos

In [18]:
le = preprocessing.LabelEncoder()

le.fit(df["VALUE_SEGMENT"])
df["VALUE_SEGMENT"] = le.transform(df["VALUE_SEGMENT"])


In [19]:
df.columns

Index(['SUSCRIBER_KEY', 'VALUE_SEGMENT', 'PREP_RECH_Q_EVT_X1',
       'PREP_RECH_Q_EVT_X2', 'PREP_RECH_Q_EVT_X3', 'TRD_M1', 'TRD_M2',
       'TRD_M3', 'PACK_DATA_Q_X1', 'PACK_DATA_Q_X2', 'PACK_DATA_Q_X3',
       'FLAG_MIG', 'PREP_RECH_AMT_W_0', 'PREP_RECH_AMT_W_1',
       'PREP_RECH_AMT_W_2', 'PREP_RECH_AMT_W_3', 'PREP_RECH_Q_EVT_W_0',
       'PREP_RECH_Q_EVT_W_1', 'PREP_RECH_Q_EVT_W_2', 'PREP_RECH_Q_EVT_W_3',
       'target', 'DATA_EXP_MEAN', 'PREP_RECH_AMT_REL_X1',
       'PREP_RECH_AMT_REL_X2', 'PREP_RECH_AMT_REL_X3', 'PACK_DATA_AMT_REL_X1',
       'PACK_DATA_AMT_REL_X2', 'PACK_DATA_AMT_REL_X3', 'PREP_RECH_Q_REL_X2',
       'PREP_RECH_Q_REL_X3', 'PACK_DATA_Q_REL_X2', 'PACK_DATA_Q_REL_X3',
       'PREP_RECH_REL_X1', 'PREP_RECH_REL_X2', 'PREP_RECH_REL_X3',
       'PACK_DATA_REL_X1', 'PACK_DATA_REL_X2', 'PACK_DATA_REL_X3',
       'DAYS_CUSTOMER', 'PERCENT_DAYS'],
      dtype='object')

In [20]:
df.columns.shape

(40,)

#### Reordenamiento de columnas

In [21]:
index_target = np.where(df.columns=="target")[0][0]
lista_orden = (df.columns.tolist()[:index_target] + df.columns.tolist()[index_target+1:] 
              + [df.columns.tolist()[index_target]])

In [22]:
df = df[lista_orden].copy()

#### Guardamos el dataframe listo para el entrenamiento

In [29]:
df.to_csv("path_datos_entrenamiento" , sep = "|",index=False)